In [ ]:
%pip install lancedb
%pip install pandas
%pip install pyarrow
%pip install ipywidgets
%pip install jupyter
%pip install sentence-transformers


In [12]:
import lancedb
import pandas as pd
import pyarrow as pa
import numpy as np

uri = "runtime_assets/vecdb"
db = lancedb.connect(uri)

In [13]:
db.table_names()

['hc100']

In [14]:
tbl = db.open_table("hc100")
tbl.count_rows()

5418

In [15]:
tbl.search(None).select(["filename"]).to_pandas().values

array([["# Table: 'AbcClassification' (Abc Classification): The classification of inventory, after ABC analysis, into three basic groups for the purpose of stock control and planning. The 3 basic categories a"],
       ["# Table: 'AbuseType' (Abuse Type): When used as a legal term in healthcare, it normally refers to actions that do not involve intentional misrepresentations in billing but which, nevert"],
       ["# Table: 'AccountStatus' (Account Status): The status of the associated account.Ex:- Current- Past due- 30 days past due- 60 days past due- 90 days past due- In collection- Delinquent"],
       ["# Table: 'AccountingBasis' (Accounting Basis): The accounting basis in which transactions that change a company's financial statements are recorded in the periods in which the events occur, rather "],
       ["# Table: 'AccountingFramework' (Accounting Framework): There are many different frameworks of accounting standards in use internationally. Each of these frameworks consists of

In [16]:
tbl.search(None).select(["content"]).to_pandas().values

array([["# Table: 'AbcClassification' (Abc Classification): The classification of inventory, after ABC analysis, into three basic groups for the purpose of stock control and planning. The 3 basic categories a\n# Columns:\n   - AbcClassificationId (integer): The unique identifier of an ABC Classification.\n   - AbcClassificationName (string): The name of an ABC Classification.\n   - AbcClassificationDescription (string): The description of an ABC Classification."],
       ["# Table: 'AbuseType' (Abuse Type): When used as a legal term in healthcare, it normally refers to actions that do not involve intentional misrepresentations in billing but which, nevert\n# Columns:\n   - AbuseTypeId (integer): The unique identifier of a type of abuse.\n   - AbuseTypeName (string): The name of a type of abuse.\n   - AbuseTypeDescription (string): The description of a type of abuse."],
       ["# Table: 'AccountStatus' (Account Status): The status of the associated account.Ex:- Current- Past due- 30 da

In [29]:
# generate an embedding request to ollama  on the sentance 
sentence = "where can I find data about allergy reactions of patients"

import requests
import json

def get_embedding(text):
    response = requests.post(
        "http://localhost:11434/api/embeddings",
        json={"model": "nomic-embed-text", "prompt": text}
    )
    return response.json()["embedding"]

question_embedding = get_embedding(sentence)
# normalize the embedding to unit length
question_embedding = question_embedding / np.linalg.norm(question_embedding)


In [17]:
from sentence_transformers import SentenceTransformer
# generate an embedding request to ollama  on the sentance 
sentence = "where can I find info about patients health insurance claims status"
# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L12-v2")
# 2. Generate embeddings for a list of sentences
sentence_embeddings = model.encode(sentence)
# 3. Normalize the embeddings to unit length
sentence_embeddings = sentence_embeddings / np.linalg.norm(sentence_embeddings)


In [18]:
df = tbl.search(sentence_embeddings, query_type="vector") \
    .select(["content"]) \
    .limit(10).to_pandas()
df.sort_values(by="content").values


array([["# Table: 'ClaimStatusType' (Claim Status Type): The status of the associated Claim.Ex:- Submitted- Open- Closed- Filed- Under Review- Contested- Uncontested- Pending arbitration- Settled/\n# Columns:\n   - ClaimStatusTypeId (integer): The unique identifier of a claim status type.\n   - ClaimStatusTypeName (string): The name of a claim status type.\n   - ClaimStatusTypeDescription (string): The description of a claim status type.",
        0.9500734806060791],
       ["# Table: 'HealthInsuranceClaim' (Health Insurance Claim): An itemized statement of healthcare services and their costs provided by a Service Provider. Claims are submitted to the insurer or managed care plan \n# Columns:\n   - ClaimId (long): The unique identifier of a Claim.\n   - PayerClaimControlNumber (integer): Unique control number of identification assigned to the claim by the payer.\n   - ServiceProviderId (integer): The unique identifier of a Service Provider.  child reference to ServiceProvider.ServiceP

In [19]:
df.sort_values(by="content").values

array([["# Table: 'ClaimStatusType' (Claim Status Type): The status of the associated Claim.Ex:- Submitted- Open- Closed- Filed- Under Review- Contested- Uncontested- Pending arbitration- Settled/\n# Columns:\n   - ClaimStatusTypeId (integer): The unique identifier of a claim status type.\n   - ClaimStatusTypeName (string): The name of a claim status type.\n   - ClaimStatusTypeDescription (string): The description of a claim status type.",
        0.9500734806060791],
       ["# Table: 'HealthInsuranceClaim' (Health Insurance Claim): An itemized statement of healthcare services and their costs provided by a Service Provider. Claims are submitted to the insurer or managed care plan \n# Columns:\n   - ClaimId (long): The unique identifier of a Claim.\n   - PayerClaimControlNumber (integer): Unique control number of identification assigned to the claim by the payer.\n   - ServiceProviderId (integer): The unique identifier of a Service Provider.  child reference to ServiceProvider.ServiceP

In [20]:
rel_tables = "\n".join(df['content'].values)

In [23]:
prompt = f"""
You are an AI assistant that answers questions about database schemas and tables.
Here are the relevant tables and columns:
{rel_tables}

Please answer the following question. First explain the purpose of the tables relevant to the question, and if needed provide a query to retrieve the data.
Question:{sentence}
"""

print(prompt)


You are an AI assistant that answers questions about database schemas and tables.
Here are the relevant tables and columns:
# Table: 'HealthInsuranceClaim' (Health Insurance Claim): An itemized statement of healthcare services and their costs provided by a Service Provider. Claims are submitted to the insurer or managed care plan 
# Columns:
   - ClaimId (long): The unique identifier of a Claim.
   - PayerClaimControlNumber (integer): Unique control number of identification assigned to the claim by the payer.
   - ServiceProviderId (integer): The unique identifier of a Service Provider.  child reference to ServiceProvider.ServiceProviderId.
   - ProviderClaimControlNumber (integer): Unique control number of identification assigned to the claim by the provider.
   - SendersClaimIdentificationNumber (integer): Unique number of identification assigned to the claim by the sender.
   - HealthClaimTypeId (integer): The unique identifier of a Health Claim Type.  child reference to HealthClai

In [ ]:
import requests
# send request to ollama
response = requests.post(
    "http://localhost:11434/api/generate",
    json={
        "model": "llama3.2",
        "prompt": prompt,
        "options": {
            "temperature": 0.1,
        },
        "stream": False
    }
)

# extract the SQL query from response
sql_query = response.json()["response"]
print("Generated SQL query:")
print(sql_query)